## init and input

In [1]:
import gensim, nltk, matplotlib
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import OnehotTransactions
from mlxtend.frequent_patterns import association_rules

In [2]:
fpaper = 'paper_dataset.txt'
fref = 'paper_reference_index.txt'
flink = 'paper_reference_links.txt'
dpaper = pd.read_csv(fpaper, sep='\t')
dref = pd.read_csv(fref, sep='\t')
dlink = pd.read_csv(flink, sep='\t')

## frequent author sets

In [3]:
arr_authors = np.array(dpaper['authors'])
# arr_authors_all = np.append(np.array(dpaper['authors']), np.array(dref['authors']))

authors_list=[]
for i in range(arr_authors.size):
    authors_list.append(arr_authors[i].split(" + "))


### merge same authors

In [4]:
all_authors = []
for authors_of_a_paper in authors_list:
    for single_author in authors_of_a_paper:
        all_authors.append(single_author)
all_authors.sort()


for index1, authors_of_a_paper in enumerate(authors_list):
    for index2, single_author in enumerate(authors_of_a_paper):
        if single_author == 'scholkopf':
            authors_list[index1][index2] = 'bernhard scholkopf'
    

In [5]:
oht = OnehotTransactions()
oht_ary = oht.fit(authors_list).transform(authors_list)
df = pd.DataFrame(oht_ary, columns=oht.columns_)

frequent_author_sets=apriori(df, min_support=0.002, use_colnames=True)
sorted_frequent_sets = frequent_author_sets.sort_values('support', ascending=False)

author_rules=association_rules(sorted_frequent_sets, metric="confidence", min_threshold=0.6)
sorted_author_rules = author_rules.sort_values(['support','lift','confidence'], ascending=[False, False, False])

## frequent paper sets

In [7]:
arr_source = np.array(dlink['source'])
arr_ref = np.array(dlink['target'])

ref_list = []
i = 0
j = 0
ref_list.append([])
for index, source in enumerate(arr_source):
    if(arr_source[index] == arr_source[i]):
        ref_list[j].append(arr_ref[index])
    else:
        i = index
        j = j + 1
        ref_list.append([])
        ref_list[j].append(arr_ref[index])

In [10]:
oht1 = OnehotTransactions()
oht_ary1 = oht1.fit(ref_list).transform(ref_list)
df1 = pd.DataFrame(oht_ary1, columns=oht1.columns_)

frequent_ref_sets=apriori(df1, min_support=0.01, use_colnames=True)

sorted_frequent_ref_sets = frequent_ref_sets.sort_values('support', ascending=False)

ref_rules=association_rules(sorted_frequent_ref_sets, metric="confidence", min_threshold=0.6)

sorted_ref_rules = ref_rules.sort_values(['lift','support','confidence'], ascending=[False, False, False])